### Imports

In [23]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
# from concurrent.futures import ThreadPoolExecutor
from tqdm.contrib.concurrent import thread_map
pd.set_option('display.max_colwidth', None)

### GET AIRLINES URLS ###

In [24]:
url = 'https://www.pilotjobsnetwork.com/'
max_urls = 5

def get_airlines_urls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href and 'jobs/' in href:
            airline_url = url + href
            airline_name = link.text.strip()
            yield airline_url, airline_name # crée un générateur

### GET INFO FROM PAGES ###

In [25]:
def get_airline_tables(airline_tuple):
    airline_url, airline_name = airline_tuple
    try:
        list_tables = pd.read_html(airline_url)
        salary = list_tables[3].iloc[2:4,1].tolist()
        # iloc iloc[2:4,2] gets you the dates, but remove the salary ... smthing to dig
        return [airline_url, airline_name] + salary
    except Exception as e:
        print(e)
        try:
            return list_tables[3]
        except NameError:
            return airline_url

### MAP IT !! ###

In [26]:
list_salaries = list(thread_map(get_airline_tables, get_airlines_urls(url)))
# print dans un excel le list_salaries

0it [00:00, ?it/s]

'ascii' codec can't encode characters in position 18-19: ordinal not in range(128)
URL can't contain control characters. '/jobs/Hello\r\n\r\nI_recently_visited_your_site__ppjn.com__from_a_Google_search_using_the_Google_chrome_browser._\r\n\r\nI_wanted_to_reach_out_as_I_noticed_it_displayed_a_not_secure_warning_just_to_the_left_of_where_the_url_is_displayed,_rather_than_the_usual_padlock_icon_you_see_on_similar_sites_to_yours._Do_you_get_that_as_well_when_you_visit_your_site?_If_you_read_on_Google' (found at least '\r')


### DF AS OUTPUT ###

In [44]:
df_salary = pd.DataFrame([salary for salary in list_salaries if type(salary) == list])
df_salary

# TO DO :
# DF needs cleaning, columns names, etc
# Check about the NaNs
# See if possible to extract currency (to dble check with meta data, maybe?)
# Add in the DF : "update dates"

# df_salary.to_excel("df_salary.xlsx")

,0,1,2,3
0,https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH,7Mar - Germany (Ma) - Eurowings GmbH,"14.540 x 13 = 189.021,43 euro TRE = +1.700 euro/month TRI = +1.600 euro/month LTC = +800 euro/month (+variable allowances for each check/training event)","8.611,37 x 13 = 111.947,81 euro (or at least +1.300 euro/month after upgrade to CPT). That means all FO with high seniority, who used to be capped at rank 11, will move to CPT rank 4 after upgrade."
1,https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines,7Mar - Hong Kong (Ma) - Greater Bay Airlines,NaN,$80000 HKD/MTH
2,https://www.pilotjobsnetwork.com/jobs/Eurowings_Europe_Ltd,6Mar - Malta (Ma) - Eurowings Europe Ltd,AT 117.252 â¬ gross/year ES 126.948 â¬ gross/year CZ 60.872 â¬ gross/year SE 811.368 SEK gross/year (This does not include 13th and 14th additional payment),AT 73.296 â¬ gross/year ES 87.132 â¬ gross/year CZ 43.470 â¬ gross/year SE 521.592 SEK gross /year (This does not include 13th and 14th additional payment)
3,https://www.pilotjobsnetwork.com/jobs/Avion_Express,6Mar - Lithuania (Ch) - Avion Express,Captains 5500 EUR basic pay + 50 EUR/BH. Loyalty bonus 2500 EUR paid twice per year (applicable from 2nd year of service),Winter basic for CPT 5000 + 50 EUR / BHÂ
4,https://www.pilotjobsnetwork.com/jobs/Corendon_Airlines,5Mar - Turkey (Ma) - Corendon Airlines,6250,5250
...,...,...,...,...
494,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,13Aug - Belgium (Fr) - FlyinGroup,NaN,NaN
495,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,12Aug - Austria (Fr) - MS AVIATION,NaN,NaN
496,https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation,12Aug - France (Re) - ESMA Aviation,NaN,NaN
497,https://www.pilotjobsnetwork.com/jobs/Compass_Airlines,9Aug - USA (Ma) - Compass Airlines,NaN,NaN


### ERRORS LIST ###

In [28]:
[salary for salary in list_salaries if type(salary) == str]

['https://www.pilotjobsnetwork.com/jobs/Air_CaraÃ¯bes_Atlantique',
 'https://www.pilotjobsnetwork.com/jobs/Hello\r\n\r\nI_recently_visited_your_site__ppjn.com__from_a_Google_search_using_the_Google_chrome_browser._\r\n\r\nI_wanted_to_reach_out_as_I_noticed_it_displayed_a_not_secure_warning_just_to_the_left_of_where_the_url_is_displayed,_rather_than_the_usual_padlock_icon_you_see_on_similar_sites_to_yours._Do_you_get_that_as_well_when_you_visit_your_site?_If_you_read_on_Google']

### DF REWORK ###

In [45]:
df_salary.columns = ["URL", "AirlineName", "CaptMax", "CaptMin"]
# df_salary.set_index("AirlineName", inplace=True)
df_salary

,URL,AirlineName,CaptMax,CaptMin
0,https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH,7Mar - Germany (Ma) - Eurowings GmbH,"14.540 x 13 = 189.021,43 euro TRE = +1.700 euro/month TRI = +1.600 euro/month LTC = +800 euro/month (+variable allowances for each check/training event)","8.611,37 x 13 = 111.947,81 euro (or at least +1.300 euro/month after upgrade to CPT). That means all FO with high seniority, who used to be capped at rank 11, will move to CPT rank 4 after upgrade."
1,https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines,7Mar - Hong Kong (Ma) - Greater Bay Airlines,NaN,$80000 HKD/MTH
2,https://www.pilotjobsnetwork.com/jobs/Eurowings_Europe_Ltd,6Mar - Malta (Ma) - Eurowings Europe Ltd,AT 117.252 â¬ gross/year ES 126.948 â¬ gross/year CZ 60.872 â¬ gross/year SE 811.368 SEK gross/year (This does not include 13th and 14th additional payment),AT 73.296 â¬ gross/year ES 87.132 â¬ gross/year CZ 43.470 â¬ gross/year SE 521.592 SEK gross /year (This does not include 13th and 14th additional payment)
3,https://www.pilotjobsnetwork.com/jobs/Avion_Express,6Mar - Lithuania (Ch) - Avion Express,Captains 5500 EUR basic pay + 50 EUR/BH. Loyalty bonus 2500 EUR paid twice per year (applicable from 2nd year of service),Winter basic for CPT 5000 + 50 EUR / BHÂ
4,https://www.pilotjobsnetwork.com/jobs/Corendon_Airlines,5Mar - Turkey (Ma) - Corendon Airlines,6250,5250
...,...,...,...,...
494,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,13Aug - Belgium (Fr) - FlyinGroup,NaN,NaN
495,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,12Aug - Austria (Fr) - MS AVIATION,NaN,NaN
496,https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation,12Aug - France (Re) - ESMA Aviation,NaN,NaN
497,https://www.pilotjobsnetwork.com/jobs/Compass_Airlines,9Aug - USA (Ma) - Compass Airlines,NaN,NaN


In [49]:
df_salary[["Update date", "City", "Name"]] = df_salary["AirlineName"].str.split(" - ", expand=True)
df_salary

,URL,AirlineName,CaptMax,CaptMin,Update date,City,Name
0,https://www.pilotjobsnetwork.com/jobs/Eurowings_GmbH,7Mar - Germany (Ma) - Eurowings GmbH,"14.540 x 13 = 189.021,43 euro TRE = +1.700 euro/month TRI = +1.600 euro/month LTC = +800 euro/month (+variable allowances for each check/training event)","8.611,37 x 13 = 111.947,81 euro (or at least +1.300 euro/month after upgrade to CPT). That means all FO with high seniority, who used to be capped at rank 11, will move to CPT rank 4 after upgrade.",7Mar,Germany (Ma),Eurowings GmbH
1,https://www.pilotjobsnetwork.com/jobs/Greater_Bay_Airlines,7Mar - Hong Kong (Ma) - Greater Bay Airlines,NaN,$80000 HKD/MTH,7Mar,Hong Kong (Ma),Greater Bay Airlines
2,https://www.pilotjobsnetwork.com/jobs/Eurowings_Europe_Ltd,6Mar - Malta (Ma) - Eurowings Europe Ltd,AT 117.252 â¬ gross/year ES 126.948 â¬ gross/year CZ 60.872 â¬ gross/year SE 811.368 SEK gross/year (This does not include 13th and 14th additional payment),AT 73.296 â¬ gross/year ES 87.132 â¬ gross/year CZ 43.470 â¬ gross/year SE 521.592 SEK gross /year (This does not include 13th and 14th additional payment),6Mar,Malta (Ma),Eurowings Europe Ltd
3,https://www.pilotjobsnetwork.com/jobs/Avion_Express,6Mar - Lithuania (Ch) - Avion Express,Captains 5500 EUR basic pay + 50 EUR/BH. Loyalty bonus 2500 EUR paid twice per year (applicable from 2nd year of service),Winter basic for CPT 5000 + 50 EUR / BHÂ,6Mar,Lithuania (Ch),Avion Express
4,https://www.pilotjobsnetwork.com/jobs/Corendon_Airlines,5Mar - Turkey (Ma) - Corendon Airlines,6250,5250,5Mar,Turkey (Ma),Corendon Airlines
...,...,...,...,...,...,...,...
494,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,13Aug - Belgium (Fr) - FlyinGroup,NaN,NaN,13Aug,Belgium (Fr),FlyinGroup
495,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,12Aug - Austria (Fr) - MS AVIATION,NaN,NaN,12Aug,Austria (Fr),MS AVIATION
496,https://www.pilotjobsnetwork.com/jobs/ESMA_Aviation,12Aug - France (Re) - ESMA Aviation,NaN,NaN,12Aug,France (Re),ESMA Aviation
497,https://www.pilotjobsnetwork.com/jobs/Compass_Airlines,9Aug - USA (Ma) - Compass Airlines,NaN,NaN,9Aug,USA (Ma),Compass Airlines


In [54]:
All_nans = df_salary.isna().sum().sum()
print(f"NaNs across the DF: {All_nans}")

CaptMax_nans = df_salary["CaptMax"].isna().sum()
print(f"Capt Max NaNs: {CaptMax_nans}")

CaptMin_nans = df_salary["CaptMin"].isna().sum()
print(f"Capt Min NaNs: {CaptMin_nans}")


NaNs across the DF: 411
Capt Max NaNs: 218
Capt Min NaNs: 193
